# Cardiac Arrythmia Classification

**Libraries Installation**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn import preprocessing
from matplotlib import pyplot as plt

In [2]:
import numpy as np

In [17]:
from sklearn.impute import SimpleImputer


**Dataset**

In [3]:
data = pd.read_csv('Dataset/cardiac_arrhythmia.csv')
output = pd.DataFrame(index=None, columns=['model','train_Rsquare', 'test_Rsquare', 'train_MSE','test_MSE'])
data.describe()

,Age,sex,height,weight,QRS_duration,P-R_interval,Q-T_interval,T_interval,P_interval,QRS,...,Amp_V6_Q_wave,Amp_V6_R_wave,Amp_V6_S_wave,Amp_V6_R'_wave,Amp_V6_S'_wave,Amp_V6_P_wave,Amp_V6_T_wave,Amp_V6_QRSA,Amp_V6_QRSTA,cardiac_arrhythmia
count,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,...,452.000000,452.000000,452.000000,452.000000,452.0,452.000000,452.000000,452.000000,452.000000,452.000000
mean,46.471239,0.550885,166.188053,68.170354,88.920354,155.152655,367.207965,169.949115,90.004425,33.676991,...,-0.278982,9.048009,-1.457301,0.003982,0.0,0.514823,1.222345,19.326106,29.473230,3.880531
std,16.466631,0.497955,37.170340,16.590803,15.364394,44.842283,33.385421,35.633072,25.826643,45.431434,...,0.548876,3.472862,2.002430,0.050118,0.0,0.347531,1.426052,13.503922,18.493927,4.407097
min,0.000000,0.000000,105.000000,6.000000,55.000000,0.000000,232.000000,108.000000,0.000000,-172.000000,...,-4.100000,0.000000,-28.600000,0.000000,0.0,-0.800000,-6.000000,-44.200000,-38.600000,1.000000
25%,36.000000,0.000000,160.000000,59.000000,80.000000,142.000000,350.000000,148.000000,79.000000,3.750000,...,-0.425000,6.600000,-2.100000,0.000000,0.0,0.400000,0.500000,11.450000,17.550000,1.000000
50%,47.000000,1.000000,164.000000,68.000000,86.000000,157.000000,367.000000,162.000000,91.000000,40.000000,...,0.000000,8.800000,-1.100000,0.000000,0.0,0.500000,1.350000,18.100000,27.900000,1.000000
75%,58.000000,1.000000,170.000000,79.000000,94.000000,175.000000,384.000000,179.000000,102.000000,66.000000,...,0.000000,11.200000,0.000000,0.000000,0.0,0.700000,2.100000,25.825000,41.125000,6.000000
max,83.000000,1.000000,780.000000,176.000000,188.000000,524.000000,509.000000,381.000000,205.000000,169.000000,...,0.000000,23.600000,0.000000,0.800000,0.0,2.400000,6.000000,88.800000,115.900000,16.000000


In [4]:
data.isnull().sum()


Age                   0
sex                   0
height                0
weight                0
QRS_duration          0
                     ..
Amp_V6_P_wave         0
Amp_V6_T_wave         0
Amp_V6_QRSA           0
Amp_V6_QRSTA          0
cardiac_arrhythmia    0
Length: 280, dtype: int64

In [5]:
(data == '?').sum()


Age                   0
sex                   0
height                0
weight                0
QRS_duration          0
                     ..
Amp_V6_P_wave         0
Amp_V6_T_wave         0
Amp_V6_QRSA           0
Amp_V6_QRSTA          0
cardiac_arrhythmia    0
Length: 280, dtype: int64

In [ ]:
# Replace '?' with NaN
import numpy as np
data['J'] = data['J'].replace('?',np.NaN)
data['HR'] = data['HR'].replace('?',np.NaN)
data['P'] = data['P'].replace('?',np.NaN)
data['T'] = data['T'].replace('?',np.NaN)
data['QRST'] = data['QRST'].replace('?',np.NaN)

**Processing Data**

In [12]:
Data_Y = data.cardiac_arrhythmia.values.ravel()
Data_X = data.drop('cardiac_arrhythmia', axis=1)

In [13]:
np.unique(Data_Y, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 14, 15, 16], dtype=int64),
 array([245,  44,  15,  15,  13,  25,   3,   2,   9,  50,   4,   5,  22],
       dtype=int64))

In [14]:
data

,Age,sex,height,weight,QRS_duration,P-R_interval,Q-T_interval,T_interval,P_interval,QRS,...,Amp_V6_Q_wave,Amp_V6_R_wave,Amp_V6_S_wave,Amp_V6_R'_wave,Amp_V6_S'_wave,Amp_V6_P_wave,Amp_V6_T_wave,Amp_V6_QRSA,Amp_V6_QRSTA,cardiac_arrhythmia
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0,0.3,3.4,12.3,49.0,10
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0,-0.1,3.9,25.4,62.8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53,1,160,70,80,199,382,154,117,-37,...,0.0,4.3,-5.0,0.0,0,0.7,0.6,-4.4,-0.5,1
448,37,0,190,85,100,137,361,201,73,86,...,0.0,15.6,-1.6,0.0,0,0.4,2.4,38.0,62.4,10
449,36,0,166,68,108,176,365,194,116,-85,...,0.0,16.3,-28.6,0.0,0,1.5,1.0,-44.2,-33.2,2
450,32,1,155,55,93,106,386,218,63,54,...,-0.4,12.0,-0.7,0.0,0,0.5,2.4,25.0,46.6,1


In [ ]:
# Dropping column due to too many NaN values
Data_X.drop(columns=['J'])

,Age,sex,height,weight,QRS_duration,P-R_interval,Q-T_interval,T_interval,P_interval,QRS,...,Amp_V6_JJ_wave,Amp_V6_Q_wave,Amp_V6_R_wave,Amp_V6_S_wave,Amp_V6_R'_wave,Amp_V6_S'_wave,Amp_V6_P_wave,Amp_V6_T_wave,Amp_V6_QRSA,Amp_V6_QRSTA
0,75,0,190,80,91,193,371,174,121,-16,...,-0.3,0.0,9.0,-0.9,0.0,0,0.9,2.9,23.3,49.4
1,56,1,165,64,81,174,401,149,39,25,...,-0.5,0.0,8.5,0.0,0.0,0,0.2,2.1,20.4,38.8
2,54,0,172,95,138,163,386,185,102,96,...,0.9,0.0,9.5,-2.4,0.0,0,0.3,3.4,12.3,49.0
3,55,0,175,94,100,202,380,179,143,28,...,0.1,0.0,12.2,-2.2,0.0,0,0.4,2.6,34.6,61.6
4,75,0,190,80,88,181,360,177,103,-16,...,-0.4,0.0,13.1,-3.6,0.0,0,-0.1,3.9,25.4,62.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53,1,160,70,80,199,382,154,117,-37,...,0.0,0.0,4.3,-5.0,0.0,0,0.7,0.6,-4.4,-0.5
448,37,0,190,85,100,137,361,201,73,86,...,-0.5,0.0,15.6,-1.6,0.0,0,0.4,2.4,38.0,62.4
449,36,0,166,68,108,176,365,194,116,-85,...,1.2,0.0,16.3,-28.6,0.0,0,1.5,1.0,-44.2,-33.2
450,32,1,155,55,93,106,386,218,63,54,...,0.2,-0.4,12.0,-0.7,0.0,0,0.5,2.4,25.0,46.6


In [18]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
z = imputer.fit_transform(Data_X)
Data_X = pd.DataFrame(data=z, columns=Data_X.columns)
